In [1]:
import pandas as pd
from sqlalchemy import create_engine
import codecs

Initialize SQLite engine

In [2]:
%%time
panlex_lite_path = "/mnt/permanent/Language/Multi/Dic/Proj/EmergVocPanLex/panlex_lite/db.sqlite"
engine = create_engine('sqlite:///{0}'.format(panlex_lite_path))

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 6.58 ms


The table __lv__ indexes language varieties

In [3]:
lv = pd.read_sql_table('lv', engine, index_col="lv")
lv.head()

,lc,vc,uid,ex,tt
lv,,,,,
1,aar,0,aar-000,1453510,Qafár af
2,abe,0,abe-000,23655591,Aln8ba8dwaw8gan
3,abk,0,abk-000,43385,аҧсуа бызшәа
4,abq,0,abq-000,43386,абаза бызшва
5,acf,0,acf-000,45822,kwéyòl


The table __ex__ indexes _expressions_

In [4]:
%%time
ex = pd.read_sql_table('ex', engine, index_col='ex')

CPU times: user 1min 33s, sys: 13.8 s, total: 1min 47s
Wall time: 2min


In [5]:
ex.head()

,lv,tt
ex,,
2,93,абхазки
3,93,английски
4,93,арабски
5,93,Бари
6,93,български


We reduce the ex table to the languages we want to keep

The __dnx__ table contains expressions coupled with meanings, and includes quality scores

In [ ]:
%%time
dnx = pd.read_sql_table('dnx', engine)

In [ ]:
dnx.head()

In [ ]:
min_score = 6

In [ ]:
langs_to_keep = ['eng', 'hun', 'deu']
lv_red = lv[lv.uid.isin(['{0}-000'.format(lang) for lang in langs_to_keep])]
langids = lv_red.index.values

create separate data frames for each language's expressions and denotations

In [ ]:
dnx_by_lang = {}
ex_by_lang = {}

In [ ]:
%%time
for lang in langids:
    print('processing {0}'.format(lang))
    if lang not in ex_by_lang:
        ex_by_lang[lang] = ex[ex.lv==lang]
    if lang not in dnx_by_lang:
        dnx_by_lang[lang] = dnx.merge(ex_by_lang[lang], left_on='ex', right_index=True)

In [ ]:
print('will process these langs: {0}'.format(
    [lv.loc[lang]['uid'][:3] for lang in langids]))
done = set()
for lang1 in langids:
    for lang2 in langids:
        lang_pair = tuple(sorted([lang1, lang2]))
        if lang1 == lang2 or lang_pair in done:
            continue
        done.add(lang_pair)
        lang1_name = lv.loc[lang1]['uid'][:3]
        lang2_name = lv.loc[lang2]['uid'][:3]
        print('doing {0}-{1}...'.format(lang1_name, lang2_name))
        ex1 = ex_by_lang[lang1]
        ex2 = ex_by_lang[lang2]
        dnx1 = dnx_by_lang[lang1]
        dnx2 = dnx_by_lang[lang2]
        
        tr = dnx1.merge(dnx2, on='mn')
        
        filtered = tr[tr.uq_x + tr.uq_y > min_score]
        print(len(tr))
        print(len(filtered))
        
        tr_sorted = tr.sort_values('uq_x', ascending=False)
        print('writing {0}-{1}...'.format(lang1_name, lang2_name))
        fn = '{0}_{1}.tsv'.format(lang1_name, lang2_name)
        
        with open(fn, 'w', encoding='utf-8') as f:
            for _, row in tr_sorted.iterrows():
                score = (row['uq_x'] + row['uq_y']) / 2
                f.write("{0}\t{1}\t{2}\t{3}\t{4}\n".format(
                    lang1_name, lang2_name, row['tt_x'], row['tt_y'], score))
                        
                        

In [ ]:
dnx1.head()

In [ ]:
dnx2.head()

In [ ]:
lv.loc[283]

In [ ]:
%%time

        #break

In [ ]:
ex[ex.index==475722]

In [ ]:
print(dnx_by_lang.keys())
print(len(dnx_by_lang[283])) # hun
print(len(dnx_by_lang[187])) # eng
print(len(dnx_by_lang[157])) # deu
dnx_by_lang[157]

In [ ]:
print(ex_by_lang.keys())
print(len(ex_by_lang[283])) # hun
print(len(ex_by_lang[187])) # eng
print(len(ex_by_lang[157])) # deu
ex_by_lang[157]

In [ ]:
lv.loc[lv.iloc[:,2].str.startswith('hun', na=True)]